In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-28 18:37:05--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.4’

minsearch.py.4      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-28 18:37:05 (24.2 MB/s) - ‘minsearch.py.4’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-28 18:37:10--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.3’

documents.json.3    100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-06-28 18:37:10 (62.7 MB/s) - ‘documents.json.3’ saved [658332/658332]



In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents =[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"], 
    keyword_fields=["course"]
)

In [10]:

index.fit(documents)

In [11]:
from openai import OpenAI

In [12]:
client = OpenAI()

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [52]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    
    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )
    return results

In [53]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUSTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question = query, context = context).strip()
    return prompt

In [54]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [57]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [59]:
rag(query)

'To run Kafka, follow these steps based on the context provided for Java and Python:\n\n### For Java\n1. Ensure you are in the project directory.\n2. Run the following command in your terminal:\n\n    ```sh\n    java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n    ```\n\n### For Python\n1. Create a virtual environment and install the required packages:\n    ```sh\n    python -m venv env\n    source env/bin/activate\n    pip install -r ../requirements.txt\n    ```\n    *Note: On Windows, the command to activate the virtual environment is slightly different:*\n    ```sh\n    env\\Scripts\\activate\n    ```\n    \n2. To use the virtual environment each time, activate it with:\n    ```sh\n    source env/bin/activate\n    ```\n    *Or on Windows:*\n    ```sh\n    env\\Scripts\\activate\n    ```\n\n3. To deactivate the virtual environment, use:\n    ```sh\n    deactivate\n    ```\n\nAlso, make sure that all Docker images are up and running if a